In [1]:
from selenium import webdriver
from pymongo import MongoClient
import re

In [2]:
from selenium.webdriver.common.keys import Keys

In [3]:
import pandas as pd

In [4]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [5]:
pd.DataFrame(columns=["Caractéristiques(Chambres,Douches et Surfaces)","Emplacements","Prix"])

,"Caractéristiques(Chambres,Douches et Surfaces)",Emplacements,Prix


In [6]:
service=Service('C:/Users/hechm/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')

In [7]:
driver=webdriver.Chrome(service=service)

In [8]:
driver.get('https://www.menzili.tn/')

In [9]:
caracteristiques=driver.find_elements(By.CLASS_NAME,'info-box')
emplacements = driver.find_elements(By.CLASS_NAME, 'block-adress')
prix = driver.find_elements(By.CLASS_NAME, 'item-box-price')

In [10]:
caracteristiques_list=[]
for l in caracteristiques:
    caracteristiques_list.append(l.text.split())
print(caracteristiques_list)
chambres_list=[]
for m in caracteristiques_list:
    for c in m:
        
        chambres_list.append(c)
douches_list=[]
for n in caracteristiques_list:
    for d in n:
        douches_list.append(d)
surfaces_list=[]
for o in caracteristiques_list:
    for s in o:
        surfaces_list.append(s)
emplacements_list=[]
for e in emplacements:
    emplacements_list.append(e.text.strip())
prix_list=[]
for p in prix:
    prix_list.append(p.text.strip())

[['3', '2', '198m²'], ['2', '1'], ['5260m²'], ['7000m²'], ['3', '3'], ['3', '2', '198m²'], ['605m²']]


In [11]:
data_tuples=list(zip(caracteristiques_list,emplacements_list,prix_list))
pd.DataFrame(data_tuples,columns=["Caractéristiques(Chambres,Douches et Surfaces)","Emplacements","Prix"])

,"Caractéristiques(Chambres,Douches et Surfaces)",Emplacements,Prix
0,"[3, 2, 198m²]","DJERBA-HOUMT SOUK, MÉDENINE",700 000 DT
1,"[2, 1]","DJERBA-HOUMT SOUK, MÉDENINE",2 200 DT / Mois
2,[5260m²],"DJERBA-HOUMT SOUK, MÉDENINE",95 DT
3,[7000m²],"DJERBA - AJIM, MÉDENINE",70 DT
4,"[3, 3]","DJERBA-HOUMT SOUK, MÉDENINE",280 DT / Nuit
5,"[3, 2, 198m²]","DJERBA-HOUMT SOUK, MÉDENINE",2 550 DT / Mois


In [12]:
client=MongoClient('mongodb://localhost:27017/')
annonces=[{"Caracteristiques(Chambres,Douches,Surface)":caracteristique,"Emplacements":emplacement,"Prix":prix} for caracteristique,emplacement,prix in zip(caracteristiques_list,emplacements_list,prix_list)]
with client:
    db=client.immobilieres
    db.annonces.insert_many(annonces)

In [13]:
client=MongoClient('mongodb://localhost:27017/')
with client:
    db=client.immobilieres
    annonces=db.annonces.find()
    annonces.next()
    annonces.next()
    annonces.next()
    annonces.next()
    annonces.next()
    annonces.next()
    annonces.rewind()
    print(list(annonces))

[{'_id': ObjectId('67598ffc44af25cd42a085a6'), 'Caracteristiques(Chambres,Douches,Surface)': ['3', '2', '198m²'], 'Emplacements': 'DJERBA-HOUMT SOUK, MÉDENINE', 'Prix': '700 000 DT'}, {'_id': ObjectId('67598ffc44af25cd42a085a7'), 'Caracteristiques(Chambres,Douches,Surface)': ['2', '1'], 'Emplacements': 'DJERBA-HOUMT SOUK, MÉDENINE', 'Prix': '2 200 DT / Mois'}, {'_id': ObjectId('67598ffc44af25cd42a085a8'), 'Caracteristiques(Chambres,Douches,Surface)': ['5260m²'], 'Emplacements': 'DJERBA-HOUMT SOUK, MÉDENINE', 'Prix': '95 DT'}, {'_id': ObjectId('67598ffc44af25cd42a085a9'), 'Caracteristiques(Chambres,Douches,Surface)': ['7000m²'], 'Emplacements': 'DJERBA - AJIM, MÉDENINE', 'Prix': '70 DT'}, {'_id': ObjectId('67598ffc44af25cd42a085aa'), 'Caracteristiques(Chambres,Douches,Surface)': ['3', '3'], 'Emplacements': 'DJERBA-HOUMT SOUK, MÉDENINE', 'Prix': '280 DT / Nuit'}, {'_id': ObjectId('67598ffc44af25cd42a085ab'), 'Caracteristiques(Chambres,Douches,Surface)': ['3', '2', '198m²'], 'Emplacements

In [14]:
driver.quit()

In [15]:
client=MongoClient('mongodb://localhost:27017/')
with client:
    db=client.immobilieres
    print(f"Il existe {db.annonces.count_documents({})} annonces immobilières")

Il existe 6 annonces immobilières
